![title](dormida.gif)

## Importamos librerias

In [1]:
#Calculos
from scipy.spatial import distance as dist
import numpy as np

#Computer vision
import cv2
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import imutils

#Deep learning
import dlib

import time
import playsound
from threading import Thread

## Creamos las funciones

### Funcion para el sonido
Lo Ejecutara en diferente tread

In [2]:
def sound_alarm(path="./alarm.wav"):
    t = Thread(target=playsound.playsound, args=(path,))
    t.deamon = True
    t.start()

### Funcion para calcular proporcion de los ojos
![algoritmos proporcion de ojos](https://www.pyimagesearch.com/wp-content/uploads/2017/04/blink_detection_plot.jpg)

In [3]:
def eye_aspect_ratio(eye):
    #Vertical eye landmarks
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    #Horizontal
    C = dist.euclidean(eye[0], eye[3])
    eye_ratio = (A+B) / (2.0 * C)
    return eye_ratio

## Definimos constantes

In [4]:
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 30
ALARM_ON = False

COUNTER = 0
EYE_COLOR = (0, 255, 0)

## Cargamos pre-train neural network
Usa una red pre-entrenada (d'libs) que se basa en un standard Histogram of Oriented Gradients + Linear SVM method
![puntos de referencia de la cara](https://www.pyimagesearch.com/wp-content/uploads/2017/04/facial_landmarks_68markup-768x619.jpg)

In [5]:
PREDICTOR_SOURCE = "./shape_predictor_68_face_landmarks.dat"
#predictor facil landmarkl
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_SOURCE)

#get the indexes of the eyes
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

## Iteramos cada frame

In [6]:
vs = VideoStream(src=0).start()
time.sleep(1.0)

In [7]:
while True:
    frame = vs.read()
    #Reducimos el tamaño
    frame = imutils.resize(frame, width = 450)
    #Convertimos a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Obtenemos las referrencias de la cara con nuestro predictor
    rects = detector(gray, 0)
    
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        #obtiene la forma de los ojos
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        #obtiene la relacion entre altura-anchura
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        
        #Usa OpenCv para obtener el cuenco de los ojos
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        
        #dibuja el cuenco
        cv2.drawContours(frame, [leftEyeHull], -1, EYE_COLOR, 1)
        cv2.drawContours(frame, [rightEyeHull], -1, EYE_COLOR, 1)

        #Si tiene el ojo cerrado, aumento el contador de frames
        if ear < EYE_AR_THRESH:
            COUNTER += 1

            if COUNTER >= EYE_AR_CONSEC_FRAMES: #Si lleva mas frames de nuestra constance cerrado hago sonar la alarma
                if not ALARM_ON:
                    ALARM_ON = True
                    sound_alarm()
                cv2.putText(frame, "ALERTA CANSANCIO!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            COUNTER = 0
            ALARM_ON = False
        
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)



    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

In [8]:
cv2.destroyAllWindows()
vs.stop()